In [1]:
    var sparkConf = new SparkConf().setAppName("LDA Example");
    var sc = new SparkContext(sparkConf);


 Load and parse the data


In [2]:
var path =  "../data/mllib/sample_lda_data.txt";
    var data = sc.textFile(path);
    var parsedData = data.map(function (s) {
       var sarray = s.trim().split(" ");
        var values = [];
        for (var i = 0; i < sarray.length; i++) {
            values[i] = parseFloat(sarray[i]);
        }
        return Vectors.dense(values);
    });


 Index documents with unique IDs


In [3]:
    var data = parsedData.zipWithIndex().map(function (doc_id) {
        return new Tuple(doc_id[1], doc_id[0]); // swap
    });
    var corpus = PairRDD.fromRDD(data).cache();


 Cluster the documents into three topics using LDA


In [4]:
    var ldaModel = new LDA().setK(3).run(corpus);


 Output topics. Each is a distribution over words (matching word count vectors)


In [5]:
    var ret = {};
    ret.vocabSize = ldaModel.vocabSize();
    ret.topics = [];

    var topics = ldaModel.topicsMatrix();
    for (var topic = 0; topic < 3; topic++) {
        var str = "";
        for (var word = 0; word < ldaModel.vocabSize(); word++) {
            str += " " + topics.apply(word, topic);
        }
        ret.topics[topic] = str;
    }
var result = ret;
    print("Learned topics (as distributions over vocab of " + result.vocabSize + " words):");
    for (var t in result.topics) {
        print("Topic " + t + ":");
        print(result.topics[t]);
    }

    sc.stop();
